In [ ]:
#############################################
##'''convert dict type jsonl data to csv'''##
###@SerajMostafa, Montana State University###
########serajmostafa@montana.edu#############
###############################<©sermost>####
#############################################

In [1]:
import json
import pandas as pd
import numpy as np

In [19]:
class jsonLister():
    def __init__(self):
        pass
    
    def t_Parse(self, tweets):
        retval = {
            "t_created_at":tweets['created_at'],
            "t_id":tweets['id'],
            "t_author_id":tweets['author_id'],
            "t_text":tweets['text'],
#             "t_lang":tweets['lang'],
            ## '''no <for> loop is implemented as public_metrics is a dictionary'''
            "t_rt_count":tweets['public_metrics']['retweet_count'],
            "t_rp_count":tweets['public_metrics']['reply_count'],
            "t_lk_count":tweets['public_metrics']['like_count'],
            "t_qt_count":tweets['public_metrics']['quote_count']
#             "annotations_start":[],
#             "annotations_end":[],
#             "url":[],
#             "display_url":[],
            }
        return retval
    
    def rt_Parse(self, data):
        retval = {
            "rt_created_at":data['created_at'],
            "rt_author_id":data['author_id'],
            "rt_lang":data['lang'],
            "rt_text":data['text'],
            "rt_id":data['id'],
#             "rt_convr_id":data['conversation_id'],
            "rt_ref_tweet_id":[],
            "rt_ref_tweet_type":[],
#             "rt_annotations_start":[],
#             "rt_annotations_end":[],
#             "rt_url":[],
#             "rt_display_url":[],
             ##'''no <for> loop is implemented as public_metrics is a dictionary'''
            "rt_rt_count":data['public_metrics']['retweet_count'],
            "rt_rp_count":data['public_metrics']['reply_count'],
            "rt_lk_count":data['public_metrics']['like_count'],
            "rt_qt_count":data['public_metrics']['quote_count']
            }
        
        if "referenced_tweets" in data:
            for items in data['referenced_tweets']:
                retval['rt_ref_tweet_id'].append(items['id'])
                retval['rt_ref_tweet_type'].append(items['type'])

#         if "entities" in dataItem:
#             if "annotations" in dataItem['entities']:
#                 for annotation in dataItem['entities']['annotations']:
#                     if "start" in annotation:
#                         retval['rt_annotations_start'].append(annotation['start'])
#                         retval['rt_annotations_end'].append(annotation['end'])

#             if "urls" in dataItem['entities']:
#                 for url in dataItem['entities']['urls']:
#                     if "url" in url:
#                         retval['rt_url'].append(url['url'])
#                     if "display_url" in url:
#                         retval['rt_display_url'].append(url['display_url'])
            
        return retval
    
    ## Only required when working with <user>    
    def user_Parse(self, user):
    ## '''this block can take data manually like in other methods, however the following dict trick gets them directly as it is in dict'''
#         retval = {
#             "username":user['username'],
#             "username":user['id'],
#             "user_tweet_count":users['public_metrics']['tweet_count'],
#             "author_id":dataItem['author_id'],    
#             }

        if "public_metrics" in user:
            for k, v in user['public_metrics'].items():
                user[k] = v
            del user['public_metrics']
        return user

    def parser(self, result):
        t_parsed_list = []
        # loop to work on 'tweets'
        for tweet in result['includes']['tweets']:
            element = self.t_Parse(tweet)
            t_parsed_list.append(element)
        t_parsed_list = pd.DataFrame(t_parsed_list)    
#         t_parsed_list.to_csv('tweets.csv', encoding='utf-8', index=False)
        
        rt_parsed_list = []
        # loop to work on 'data list'
        for item in result['data']:
            element = self.rt_Parse(item)
            rt_parsed_list.append(element)
        ## used <explode()> to create new rows for each id
        rt_parsed_list = pd.DataFrame(rt_parsed_list).explode(column='rt_ref_tweet_id',ignore_index=True)
        
    ## only required when working with <user> fields
        user_parsed_list = []
        # loop to work on 'users'
        for user in result['includes']['users']:
            element = self.user_Parse(user)
            user_parsed_list.append(element)
        user_parsed_list = pd.DataFrame(user_parsed_list)
#         user_parsed_list.to_csv('users.csv', encoding='utf-8', index=False) # 

        global a, b, c
        a,b,c = rt_parsed_list, t_parsed_list, user_parsed_list
        ## Merge the tables now based on ids' <author_id, id> etc.
        table = rt_parsed_list.merge(t_parsed_list, left_on='rt_ref_tweet_id', right_on='t_id', how='left')
#         table[['rt_id','rt_author_id','rt_ref_tweet_id','t_id','rt_author_id']].to_csv('test5_jan.csv', encoding='utf-8', index=False) ## this is for testing only
        
        
        table = table.merge(user_parsed_list, left_on='rt_author_id', right_on='id', how='left')
        
#         table.to_csv('test_jan.csv', encoding='utf-8', index=False) ## this is for testing only
#         table[['rt_author_id','t_author_id','t_rt_count']].to_csv('test3_jan.txt', encoding='utf-8', index=False) ## this is for testing only
        
        
#         table.rt_ref_tweet_type = table.rt_ref_tweet_type.apply(','.join)
        table.rt_created_at= pd.to_datetime(table.rt_created_at)
        table.t_created_at= pd.to_datetime(table.t_created_at)
        table.created_at= pd.to_datetime(table.created_at)
#         table.to_csv('dec19_01-31_000000-235959.csv', encoding='utf-8', index=False)
        
        ## Other options [optional]
#         table.rt_created_at= table.rt_created_at.apply(lambda x: x.split('.')[0].replace('T',' '))
#         table = table.merge(user_parsed_list, how="left", left_on='tt_author_id', right_on='id')
#         table = table.drop_duplicates("tt_text", keep='first')
            
#         print(t_parsed_list)
#         print("-"*100)
#         print(rt_parsed_list)
#         print("="*100)
#         print(user_parsed_list)
        
        ### see customized prints to find match to do the merging    
#         print(t_parsed_list[['t_id','t_author_id']])
#         print(rt_parsed_list[['rt_ref_tweet_id','rt_id']])        #,'rt_author_id'
#         print(user_parsed_list[['id','followers_count', 'following_count']])
       
        return table

In [20]:
jl = jsonLister()

In [31]:
with open('jan20_24-27_000000-235959.jsonl', 'r', encoding="utf8") as file:
    res = []
    for json_str in file:
        result = json.loads(json_str)
        res.append(jl.parser(result))

In [32]:
df = pd.concat(res, ignore_index=True)

In [33]:
pd.set_option('display.max_columns', None)

In [34]:
# df
## df[df["rt_created_at"].between('2020-02-04 20:59:59','2020-02-04 23:59:59')]

In [35]:
# df.to_csv('jan20_24-27_000000-235959.csv', encoding='utf-8', index=False)

In [ ]:
# df_w = df[df["rt_created_at"].between('2019-12-01 00:00:00','2019-12-01 23:59:59')]#df[(df['rt_created_at']> "2019-12-01") & (df['rt_created_at']< "2019-12-03")]

In [ ]:
df_uw = df[['rt_author_id','t_author_id','rt_created_at']]

In [ ]:
df_uw

In [ ]:
# df_uw.to_csv('dec_uw_1x24.tsv',index=False, sep='\t') #.tsv